In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree

data = pd.read_csv('Popular_Baby_Names.csv', dtype = {"Child's First Name": str})

In [2]:
data.iloc[0,:]

Year of Birth              2011
Gender                   FEMALE
Ethnicity              HISPANIC
Child's First Name    GERALDINE
Count                        13
Rank                         75
Name: 0, dtype: object

In [3]:
data

,Year of Birth,Gender,Ethnicity,Child's First Name,Count,Rank
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53
...,...,...,...,...,...,...
57577,2014,MALE,WHITE NON HISPANIC,Yousef,18,94
57578,2014,MALE,WHITE NON HISPANIC,Youssef,24,88
57579,2014,MALE,WHITE NON HISPANIC,Yusuf,16,96
57580,2014,MALE,WHITE NON HISPANIC,Zachary,90,39


In [4]:
data['Gender'] = (data['Gender'] == 'FEMALE').astype(int)
data['Ethnicity'] = data['Ethnicity'].apply(lambda x: str(x).split()[0])

In [5]:
data = data.rename(columns = {"Child's First Name":"Name"})

In [6]:
Ethnicity = data.loc[:,'Ethnicity'].drop_duplicates()
Ethnicity

0      HISPANIC
206       WHITE
547       ASIAN
698       BLACK
Name: Ethnicity, dtype: object

In [7]:
for i in range(data.shape[0]):
    if str(data.loc[i,'Ethnicity']) == 'HISPANIC':
        data.loc[i,'Ethnicity'] = 0
    if str(data.loc[i,'Ethnicity']) == 'WHITE':
        data.loc[i,'Ethnicity'] = 1
    if str(data.loc[i,'Ethnicity']) == 'ASIAN':
        data.loc[i,'Ethnicity'] = 2
    else:
        data.loc[i,'Ethnicity'] = 3
data

,Year of Birth,Gender,Ethnicity,Name,Count,Rank
0,2011,1,3,GERALDINE,13,75
1,2011,1,3,GIA,21,67
2,2011,1,3,GIANNA,49,42
3,2011,1,3,GISELLE,38,51
4,2011,1,3,GRACE,36,53
...,...,...,...,...,...,...
57577,2014,0,3,Yousef,18,94
57578,2014,0,3,Youssef,24,88
57579,2014,0,3,Yusuf,16,96
57580,2014,0,3,Zachary,90,39


In [8]:
name = data['Name']
name.drop_duplicates(inplace = True)
name.reset_index(drop = True, inplace = True)
name_frame = pd.DataFrame(name)
name_frame['Index'] = 0 
for i in range(name_frame.shape[0]):
    name_frame.loc[i, 'Index'] = i
name_frame

,Name,Index
0,GERALDINE,0
1,GIA,1
2,GIANNA,2
3,GISELLE,3
4,GRACE,4
...,...,...
3206,Mahamadou,3206
3207,Heavenly,3207
3208,Emmanuella,3208
3209,Zaria,3209


In [9]:
data['Name'] = data['Name'].map(lambda x: name_frame.loc[name_frame['Name'] == x, 'Index'].iloc[0] if not name_frame.loc[name_frame['Name'] == x].empty else x)

data

,Year of Birth,Gender,Ethnicity,Name,Count,Rank
0,2011,1,3,0,13,75
1,2011,1,3,1,21,67
2,2011,1,3,2,49,42
3,2011,1,3,3,38,51
4,2011,1,3,4,36,53
...,...,...,...,...,...,...
57577,2014,0,3,2631,18,94
57578,2014,0,3,2307,24,88
57579,2014,0,3,2700,16,96
57580,2014,0,3,1745,90,39


In [10]:
X = data.drop(columns = ['Year of Birth', 'Rank'])
y = data['Year of Birth']
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.05)

In [11]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [12]:
predictions = model.predict(X_test)
score = accuracy_score(y_test, predictions)
score

0.7895833333333333

In [13]:
class_names = sorted(y.unique().astype(str))
tree.export_graphviz(model, out_file='PBN.dot',
                     feature_names=['Year of Birth', 'Gender', 'Ethnicity', 'Name'],
                     class_names=class_names,
                     label='all',
                     rounded=True,
                     filled=True)

In [14]:
# joblib.dump(model, 'PBN.joblib')